<a href="https://colab.research.google.com/github/amankumawat-567/Article-Recognition/blob/main/ArticleRecognitionGer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This is the transformer based approach in which we have considered each article number as a separate class,
# in this the preprocessed short and long text each goes through a embedding layer and then through a multi
# attention head followed by a normalisation and the both goes through cross attention and a dense layer and
# finally enters the final classification layer and a softmax,This model outputs the probability of which
# articles and we'll a particular threshold probability which will determine which articles are likely
# associated with the text.

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


 #  **Loading DATABASE (Mapped Positions)**

In [ ]:
data = pd.read_excel('/content/drive/MyDrive/ArticleRecognition/Dataset/ArticleExtraction.xlsx', sheet_name='Mapped Positions')

In [ ]:
data.sample(5)

,ProjectLVId,OzNumber,PositionShortText,PositionLongTextTitle,PositionLongText,ProjectArticleId,ProjectStadiumId,ArticleNumber,ArticleSortName,BasePrice,...,SequenceNumber,Comment,RegiePrice,LongText,Preprocessed_PositionShortText,Preprocessed_LongText,MergedPositions,Preprocessed_ArticleSortName,Preprocessed_ArticleDescription,MergedArticles
1233,NaN,NaN,NaN,NaN,NaN,22991,9092,80-ASP-0050,ASPHALT,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,;,asphalt,tragschicht 0 / 32 c ka 70 b 70 / 100,asphalt;tragschicht 0 / 32 c ka 70 b 70 / 100
2766,347153.0,1. 2. 3,"Klassenraumfassade, Ostflügel 1. und 2.OG",Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",25923,9992,23000015,OSB315,3988.508,...,2.1.3,NaN,NaN,Die Elementierung der Wände ist durch den AN z...,klassenraumfassade ostflügel 1 2og,elementierung wände wählen mögliche rückverank...,klassenraumfassade ostflügel 1 2og;elementieru...,osb315,osb3 din en 13986 / en 300 15 mm formaldehydfr...,osb315;osb3 din en 13986 / en 300 15 mm formal...
1425,347170.0,1. 3. 6,Bekleidung Wandflächen - Fassadentafeln - FH5,Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",25925,9992,24900100,ETFUGENBAND,1765.094,...,2.1.5,NaN,NaN,Liefern und montieren von Fassadenbekleidung m...,bekleidung wandflächen fassadentafeln fh5,liefern montieren fassadenbekleidung faserzeme...,bekleidung wandflächen fassadentafeln fh5;lief...,etfugenband,eternitfugenband alu schwarz 25x007 m,etfugenband;eternitfugenband alu schwarz 25x007 m
2570,NaN,NaN,NaN,NaN,NaN,23099,9093,12-HLZ-0010,HLZ,0.000,...,NaN,NaN,NaN,NaN,NaN,NaN,;,hlz,planfüllziegel pfzt12 df 07 / 8 373 / 24 / 249...,hlz;planfüllziegel pfzt12 df 07 / 8 373 / 24 /...
139,319945.0,01.13.0040,"Fundamenterder,Bandstahl,30/3,5 mm",Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",22974,9092,13-ERD-0010,ERDER,131.250,...,2.1,NaN,NaN,"Fundamenterder, einschl. allem Zubehör, in Fun...",fundamenterder bandstahl30 / 35 mm,fundamenterder einschl zubehör fundament wands...,fundamenterder bandstahl30 / 35 mm;fundamenter...,erder,erdungsband verzinkt z500 flachband 30 x 35 mm,erder;erdungsband verzinkt z500 flachband 30 x...


In [ ]:
df = data.copy()

In [ ]:
df

,ProjectLVId,OzNumber,PositionShortText,PositionLongTextTitle,PositionLongText,ProjectArticleId,ProjectStadiumId,ArticleNumber,ArticleSortName,BasePrice,...,SequenceNumber,Comment,RegiePrice,LongText,Preprocessed_PositionShortText,Preprocessed_LongText,MergedPositions,Preprocessed_ArticleSortName,Preprocessed_ArticleDescription,MergedArticles
0,35340.0,53.38.05.033110,"Zulage Schweißung flache Ecke, PVC-P, SFD 24",Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",1742,1103,K141011032000,fugenband,14.3,...,1.1,NaN,NaN,Zulage für die Schweißung einer flachen Ecke e...,zulage schweißung flache ecke pvcp sfd 24,zulage schweißung flachen ecke innenliegenden ...,zulage schweißung flache ecke pvcp sfd 24;zula...,fugenband,schweißung flache ecke pvcp sfd 24,fugenband;schweißung flache ecke pvcp sfd 24
1,35341.0,53.38.05.033120,"Zulage Schweißung senkrechte Ecke, PVC-P, SFD 24",Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",1743,1103,K141011032100,fugenband,22.9,...,1.1,NaN,NaN,Zulage für die Schweißung einer senkrechten Ec...,zulage schweißung senkrechte ecke pvcp sfd 24,zulage schweißung senkrechten ecke innenliegen...,zulage schweißung senkrechte ecke pvcp sfd 24;...,fugenband,schweißgsenkrechte ecke pvcp sfd 24,fugenband;schweißgsenkrechte ecke pvcp sfd 24
2,35342.0,53.38.05.033130,"Zulage Schweißung flaches T-Stück, PVC-P, SFD 24",Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",1744,1103,K141011032200,fugenband,14.7,...,1.1,NaN,NaN,Zulage für die Schweißung eines flachen T-Stüc...,zulage schweißung flaches tstück pvcp sfd 24,zulage schweißung flachen tstücks innenliegend...,zulage schweißung flaches tstück pvcp sfd 24;z...,fugenband,schweißgflaches tstück pvcp sfd 24,fugenband;schweißgflaches tstück pvcp sfd 24
3,35343.0,53.38.05.033140,"Zulage Schweißung senkrechtes T-Stück, PVC-P, ...",Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",1745,1103,K141011032300,fugenband,26.0,...,1.1,NaN,NaN,Zulage für die Schweißung eines senkrechten T-...,zulage schweißung senkrechtes tstück pvcp sfd 24,zulage schweißung senkrechten tstücks innenlie...,zulage schweißung senkrechtes tstück pvcp sfd ...,fugenband,schweißgsenkr tstück pvcp sfd 24,fugenband;schweißgsenkr tstück pvcp sfd 24
4,35344.0,53.38.05.033150,"Zulage Schweißung flache Kreuzung, PVC-P, SFD 24",Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",1746,1103,K141011032400,fugenband,27.1,...,1.1,NaN,NaN,Zulage für die Schweißung einer flachen Kreuzu...,zulage schweißung flache kreuzung pvcp sfd 24,zulage schweißung flachen kreuzung innenliegen...,zulage schweißung flache kreuzung pvcp sfd 24;...,fugenband,schweißgflache kreuzung pvcp sfd 24,fugenband;schweißgflache kreuzung pvcp sfd 24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3089,344554.0,1.1.1.0010,Baustelleneinrichtung,Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",24972,9970,MK,NaN,100.0,...,4.3,NaN,NaN,"Einrichten, Unterhalten, Abbauen der Baustelle...",baustelleneinrichtung,einrichten unterhalten abbauen baustelleneinri...,baustelleneinrichtung;einrichten unterhalten a...,NaN,materialcontainer,;materialcontainer
3090,344748.0,1.4.3.0010,Verbau (Box/Gleitschienenverbau),Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",25047,9971,3303,Verbau Graben bis,1400.0,...,4.1,NaN,NaN,Großflächenverbau (Box/Gleitschienenverbau) li...,verbau box / gleitschienenverbau,großflächenverbau box / gleitschienenverbau li...,verbau box / gleitschienenverbau ;großflächenv...,verbau graben,verbau graben 3 m tiefe,verbau graben ;verbau graben 3 m tiefe
3091,344748.0,1.4.3.0010,Verbau (Box/Gleitschienenverbau),Long Text 1,"<div style=""font-size:12pt;font-family:Arial;""...",25047,9971,3303,Verbau Graben bis,1400.0,...,2.1,NaN,NaN,Großflächenverbau (Box/Gleitschienenverbau) li...,verbau box / gleitschienenverbau,großflächenverbau box / gleitschienenverbau li...,verbau box / gleitschienenver

#**Extracting Useful columns**

In [ ]:
df = df[['PositionShortText','LongText','ArticleNumber','Quantity']]

In [ ]:
df.head()

,PositionShortText,LongText,ArticleNumber,Quantity
0,"Zulage Schweißung flache Ecke, PVC-P, SFD 24",Zulage für die Schweißung einer flachen Ecke e...,K141011032000,1.0
1,"Zulage Schweißung senkrechte Ecke, PVC-P, SFD 24",Zulage für die Schweißung einer senkrechten Ec...,K141011032100,1.0
2,"Zulage Schweißung flaches T-Stück, PVC-P, SFD 24",Zulage für die Schweißung eines flachen T-Stüc...,K141011032200,1.0
3,"Zulage Schweißung senkrechtes T-Stück, PVC-P, ...",Zulage für die Schweißung eines senkrechten T-...,K141011032300,1.0
4,"Zulage Schweißung flache Kreuzung, PVC-P, SFD 24",Zulage für die Schweißung einer flachen Kreuzu...,K141011032400,1.0


#**Text Preprocessing**

## 1.Handling Missing Data

In [ ]:
# Identify missing data
missing_data = df.isnull().any(axis=1)

# Filter out rows with missing data
data_cleaned = df[~missing_data]

# Drop the index associated with the rows containing missing data
data_cleaned = data_cleaned.reset_index(drop=True)

In [ ]:
data_cleaned.isnull().sum()

PositionShortText    0
LongText             0
ArticleNumber        0
Quantity             0
dtype: int64

##2.Lowercasing the Text

In [ ]:
 # Convert all string columns to lowercase
data_cleaned = data_cleaned.applymap(lambda x: x.lower() if isinstance(x, str) else x)

## 3.Remove punctuation and special characters

In [ ]:
import re

# Function to remove punctuation and special characters from text
def remove_punctuation_special_characters(text):
    # Define regex pattern to match any character that is not a word character or whitespace
    pattern = r'[^\w\säöüßÄÖÜ]'  # Including German special characters ä, ö, ü, ß, Ä, Ö, and Ü

    # Use regex to substitute all non-word characters and non-whitespace characters with an empty string
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

# Remove punctuation and special characters from 'LongText' column
data_cleaned['LongText'] = data_cleaned['LongText'].apply(remove_punctuation_special_characters)

# Remove punctuation and special characters from 'PositionShortText' column
data_cleaned['PositionShortText'] = data_cleaned['PositionShortText'].apply(remove_punctuation_special_characters)



## 4.Removing Stopwords

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Download stopwords list for German if not already downloaded
nltk.download('stopwords')
nltk.download('punkt')

# Get the set of German stopwords
stop_words_german = set(stopwords.words('german'))

# Function to remove stopwords from text
def remove_stopwords_german(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words_german]
    return ' '.join(filtered_tokens)

# Remove stopwords from 'LongText' column
data_cleaned['LongText'] = data_cleaned['LongText'].apply(remove_stopwords_german)

# Remove stopwords from 'PositionShortText' column
data_cleaned['PositionShortText'] = data_cleaned['PositionShortText'].apply(remove_stopwords_german)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## 5.Lemmetization

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# !pip install pattern

In [ ]:
from pattern.de import lemma

# Function to lemmatize text in German
def lemmatize_text_german(text):
    tokens = text.split()  # Split text into tokens
    lemmatized_tokens = []
    for token in tokens:
        try:
            lemmatized_token = lemma(token)
            lemmatized_tokens.append(lemmatized_token)
        except StopIteration:
            # Handle StopIteration exception
            # You can choose to ignore the token or handle it differently
            pass
    lemmatized_text = ' '.join(lemmatized_tokens)  # Join lemmatized tokens back into text
    return lemmatized_text

# Lemmatize 'LongText' column in German
data_cleaned['LongText'] = data_cleaned['LongText'].apply(lemmatize_text_german)

# Lemmatize 'PositionShortText' column in German
data_cleaned['PositionShortText'] = data_cleaned['PositionShortText'].apply(lemmatize_text_german)



## 6.Tokenization

In [ ]:
import nltk
from nltk.tokenize import word_tokenize

# Download NLTK resources for German language if not already downloaded
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

# Tokenize 'LongText' column in German
data_cleaned['LongText_tokens'] = data_cleaned['LongText'].apply(lambda x: word_tokenize(x, language='german'))

# Tokenize 'PositionShortText' column in German
data_cleaned['PositionShortText_tokens'] = data_cleaned['PositionShortText'].apply(lambda x: word_tokenize(x, language='german'))


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
data_cleaned


,PositionShortText,LongText,ArticleNumber,Quantity,LongText_tokens,PositionShortText_tokens
0,zulagen schweißungen flachen ecken pvcpen sfde...,zulagen schweißungen flachen ecken innenliegen...,k141011032000,1.0,"[zulagen, schweißungen, flachen, ecken, innenl...","[zulagen, schweißungen, flachen, ecken, pvcpen..."
1,zulagen schweißungen senkrechen ecken pvcpen s...,zulagen schweißungen senkrechen ecken innenlie...,k141011032100,1.0,"[zulagen, schweißungen, senkrechen, ecken, inn...","[zulagen, schweißungen, senkrechen, ecken, pvc..."
2,zulagen schweißungen flachesen tstücken pvcpen...,zulagen schweißungen flachen tstücksen innenli...,k141011032200,1.0,"[zulagen, schweißungen, flachen, tstücksen, in...","[zulagen, schweißungen, flachesen, tstücken, p..."
3,zulagen schweißungen senkrechtesen tstücken pv...,zulagen schweißungen senkrechen tstücksen inne...,k141011032300,1.0,"[zulagen, schweißungen, senkrechen, tstücksen,...","[zulagen, schweißungen, senkrechtesen, tstücke..."
4,zulagen schweißungen flachen kreuzungen pvcpen...,zulagen schweißungen flachen kreuzungen innenl...,k141011032400,1.0,"[zulagen, schweißungen, flachen, kreuzungen, i...","[zulagen, schweißungen, flachen, kreuzungen, p..."
...,...,...,...,...,...,...
2003,baustelleneinrichtungen,einrichten unterhalten abbauen baustelleneinri...,mk,2.0,"[einrichten, unterhalten, abbauen, baustellene...",[baustelleneinrichtungen]
2004,verbauen boxgleitschienenverbauen,großflächenverbauen boxgleitschienenverbauen l...,3303,2000.0,"[großflächenverbauen, boxgleitschienenverbauen...","[verbauen, boxgleitschienenverbauen]"
2005,verbauen boxgleitschienenverbauen,großflächenverbauen boxgleitschienenverbauen l...,3303,2000.0,"[großflächenverbauen, boxgleitschienenverbauen...","[verbauen, boxgleitschienenverbauen]"
2006,baustelleneinrichtungen,einrichten unterhalten abbauen baustelleneinri...,bc,2.0,"[einrichten, unterhalten, abbauen, baustellene...",[baustelleneinrichtungen]


In [ ]:
# Save the DataFrame to an Excel file
# data_cleaned.to_excel('/content/drive/MyDrive/data_cleaned.xlsx', index=False)

data_cleaned = pd.read_excel('/content/drive/MyDrive/data_cleaned.xlsx')

In [ ]:
data_cleaned

,PositionShortText,LongText,ArticleNumber,Quantity,LongText_tokens,PositionShortText_tokens
0,zulagen schweißungen flachen ecken pvcpen sfde...,zulagen schweißungen flachen ecken innenliegen...,k141011032000,1.0,"['zulagen', 'schweißungen', 'flachen', 'ecken'...","['zulagen', 'schweißungen', 'flachen', 'ecken'..."
1,zulagen schweißungen senkrechen ecken pvcpen s...,zulagen schweißungen senkrechen ecken innenlie...,k141011032100,1.0,"['zulagen', 'schweißungen', 'senkrechen', 'eck...","['zulagen', 'schweißungen', 'senkrechen', 'eck..."
2,zulagen schweißungen flachesen tstücken pvcpen...,zulagen schweißungen flachen tstücksen innenli...,k141011032200,1.0,"['zulagen', 'schweißungen', 'flachen', 'tstück...","['zulagen', 'schweißungen', 'flachesen', 'tstü..."
3,zulagen schweißungen senkrechtesen tstücken pv...,zulagen schweißungen senkrechen tstücksen inne...,k141011032300,1.0,"['zulagen', 'schweißungen', 'senkrechen', 'tst...","['zulagen', 'schweißungen', 'senkrechtesen', '..."
4,zulagen schweißungen flachen kreuzungen pvcpen...,zulagen schweißungen flachen kreuzungen innenl...,k141011032400,1.0,"['zulagen', 'schweißungen', 'flachen', 'kreuzu...","['zulagen', 'schweißungen', 'flachen', 'kreuzu..."
...,...,...,...,...,...,...
2003,baustelleneinrichtungen,einrichten unterhalten abbauen baustelleneinri...,mk,2.0,"['einrichten', 'unterhalten', 'abbauen', 'baus...",['baustelleneinrichtungen']
2004,verbauen boxgleitschienenverbauen,großflächenverbauen boxgleitschienenverbauen l...,3303,2000.0,"['großflächenverbauen', 'boxgleitschienenverba...","['verbauen', 'boxgleitschienenverbauen']"
2005,verbauen boxgleitschienenverbauen,großflächenverbauen boxgleitschienenverbauen l...,3303,2000.0,"['großflächenverbauen', 'boxgleitschienenverba...","['verbauen', 'boxgleitschienenverbauen']"
2006,baustelleneinrichtungen,einrichten unterhalten abbauen baustelleneinri...,bc,2.0,"['einrichten', 'unterhalten', 'abbauen', 'baus...",['baustelleneinrichtungen']


In [ ]:
from keras.preprocessing.text import Tokenizer

# Initialize a tokenizer
tokenizer = Tokenizer()

# Fit the tokenizer on the LongText_tokens and PositionShortText_tokens
all_tokens = data_cleaned['LongText_tokens'].tolist() + data_cleaned['PositionShortText_tokens'].tolist()
tokenizer.fit_on_texts(all_tokens)

# Convert text to sequences of numbers
data_cleaned['LongText_sequences'] = tokenizer.texts_to_sequences(data_cleaned['LongText_tokens'])
data_cleaned['PositionShortText_sequences'] = tokenizer.texts_to_sequences(data_cleaned['PositionShortText_tokens'])


##**Encoding The Article Number**


In [ ]:
data_cleaned['ArticleNumber'] = data_cleaned['ArticleNumber'].astype(str)


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit LabelEncoder on the article numbers and transform them
data_cleaned['ArticleNumber_encoded'] = label_encoder.fit_transform(data_cleaned['ArticleNumber'])

# Print the mapping of original article numbers to encoded values
#print("Mapping of original article numbers to encoded values:")
#for original_value, encoded_value in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)):
#    print(f"{original_value}: {encoded_value}")


In [ ]:
encoded_variable_counts = data_cleaned['ArticleNumber_encoded'].value_counts()
TotalArticles = encoded_variable_counts.shape[0]
print(f"Unique Articles : {TotalArticles}")

Unique Articles : 306


In [ ]:
# Group the DataFrame by 'ArticleNumber_encoded' and aggregate the indexes for each group
indexes_by_article_number = data_cleaned.groupby('ArticleNumber_encoded').apply(lambda x: list(x.index))

# Print the indexes for each article number
#for article_number, indexes in indexes_by_article_number.items():
#    print(f"Article Number {article_number}: Indexes {indexes}")


In [ ]:
 # Extract long text sequences and position short text sequences from the DataFrame
long_text_sequences = data_cleaned['LongText_sequences'].tolist()
position_short_text_sequences = data_cleaned['PositionShortText_sequences'].tolist()


In [ ]:
# Convert each integer sequence into a list of lists
tokenized_long_text_sequences = [[token] for token in long_text_sequences]
tokenized_position_short_text_sequences = [[token] for token in position_short_text_sequences]



In [ ]:
from collections import Counter

# Assuming tokenized_long_text_sequences and tokenized_position_short_text_sequences are lists of tokenized sequences
# Remove the extra brackets around each sequence
tokenized_long_text = tokenized_long_text_sequences  # List of tokenized long text sequences
tokenized_short_text = tokenized_position_short_text_sequences  # List of tokenized short text sequences

# Concatenate tokenized sequences
all_tokenized_texts = tokenized_long_text + tokenized_short_text

# Flatten the list of tokenized sequences and handle nested lists
all_tokens = [token for sequence in all_tokenized_texts for token_list in sequence for token in token_list]

# Count the frequency of each token
token_counts = Counter(all_tokens)

# Calculate the vocabulary size
vocabulary_size = len(token_counts)

print("Vocabulary size:", vocabulary_size)


Vocabulary size: 2140


## Decoder

In [ ]:
# Decode the encoded article numbers back to their original values
decoded_article_numbers = label_encoder.inverse_transform(data_cleaned['ArticleNumber_encoded'])

# Print the decoded article numbers
print("Decoded article numbers:")
print(decoded_article_numbers)


Decoded article numbers:
['k141011032000' 'k141011032100' 'k141011032200' ... '3303' 'bc' 'bc']


# Train-Test Split of the data

In [ ]:
from sklearn.model_selection import train_test_split

# Define features (X) and target variable (y)
X = data_cleaned[['LongText_sequences', 'PositionShortText_sequences' ]]  # Features
y = data_cleaned[['ArticleNumber_encoded','Quantity']]  # Target variable (replace 'Target_Column' with your actual target column name)

# Split the data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shapes of the train and test sets
print("Train set shape:", X_train.shape, y_train.shape)
print("Test set shape:", X_test.shape, y_test.shape)


Train set shape: (1606, 2) (1606, 2)
Test set shape: (402, 2) (402, 2)


In [ ]:
X_train.iloc[0:6]


,LongText_sequences,PositionShortText_sequences
1249,"[38, 75, 551, 205, 231, 144, 167, 168, 162, 16...","[554, 205, 1, 1216]"
261,"[124, 45, 125, 126, 127, 7, 8, 10, 7, 86, 17, ...","[120, 222, 6, 428]"
607,"[556, 619, 383, 587, 52]","[620, 1717, 990]"
973,"[124, 45, 125, 126, 127, 7, 8, 10, 7, 86, 17, ...","[120, 154, 50]"
240,"[38, 75, 399, 522, 205, 136, 7, 8, 221, 16, 66...","[381, 205, 1, 650]"
819,"[38, 75, 218, 266, 71, 8, 221, 207, 55, 196, 1...","[210, 225, 136, 400]"


In [ ]:
y_train.head()

,ArticleNumber_encoded,Quantity
1249,183,0.21
261,196,5022.00
607,266,1.00
973,218,6820.00
240,182,3.00


##Storing each of it as separate variables

In [ ]:
# Store X_train and X_test columns as separate variables
X_train_LongText_sequences = X_train['LongText_sequences']
X_train_PositionShortText_sequences = X_train['PositionShortText_sequences']

X_test_LongText_sequences = X_test['LongText_sequences']
X_test_PositionShortText_sequences = X_test['PositionShortText_sequences']

# Store 'Quantity' column for train and test sets
y_train_quantity = y_train['Quantity']
y_test_quantity = y_test['Quantity']

# Store 'ArticleNumber' column for train and test sets
y_train_article_number = y_train['ArticleNumber_encoded']
y_test_article_number = y_test['ArticleNumber_encoded']


# Classification Regression Model

## SELF-ATTENTION Calculation

In [ ]:
def scaled_dot_product_attention(q, k, v):
    """
    Args:
    q: query shape == (..., seq_len_q, depth) # NOTE: depth=dk
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable to (..., seq_len_q, seq_len_k). Defaults to None.

    Returns:
    output, attention_weights
    """
    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
    # scale matmul_qk. underroot d_model i.e. underroot(100)
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
    # softmax is normalized on the last axis (seq_len_k) so that the scores add up to 1.
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)
    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights

## MultiHeadAttention Calculation

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model  # typically 512

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        """Split the last dimension into (num_heads, depth).
        Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
        """
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
        # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

        concat_attention = tf.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights

## Attention Layers

In [ ]:
class AttentionLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, rate=0.1):
        super(AttentionLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)
        self.cross_mha = MultiHeadAttention(d_model, num_heads)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)

    def call(self, x1, x2, training):
        attn1_output, _ = self.mha1(x1, x1, x1)
        attn1_output = self.dropout1(attn1_output, training=training)
        outx1 = self.layernorm1(x1 + attn1_output)

        attn2_output, _ = self.mha2(x2, x2, x2)
        attn2_output = self.dropout2(attn2_output, training=training)
        outx2 = self.layernorm2(x2 + attn2_output)

        attn3_output, _ = self.cross_mha(outx1, outx1, outx2)
        attn3_output = self.dropout3(attn3_output, training=training)
        out = self.layernorm3(outx2 + attn3_output)

        return out

In [ ]:
class MultiLevelAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, input_vocab_size, rate=0.1):
        super(MultiLevelAttention, self).__init__()
        self.d_model = d_model

        self.embedding1 = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.embedding2 = tf.keras.layers.Embedding(input_vocab_size, d_model)

        self.attention_layers = AttentionLayer(d_model, num_heads, rate)

    def call(self, x1, x2, training):

        x1 = self.embedding1(x1)
        x2 = self.embedding2(x2)

        x = self.attention_layers(x1, x2, training)

        return x

## Dual Feed Forward Layer

In [ ]:
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model)
    ])

class DualFeedForwardLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, dff, rate=0.1):
        super(DualFeedForwardLayer, self).__init__()

        self.ffn1 = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)

    def call(self, x, training):
        ffn1_output = self.ffn1(x)
        ffn1_output = self.dropout1(ffn1_output, training=training)
        out1 = self.layernorm1(x + ffn1_output)

        return out1

## Classification Regression Transformer

In [ ]:
class Transformer(tf.keras.Model):
    def __init__(self, d_model, num_heads, dff, input_vocab_size,
               target_categories, rate=0.1):
        super(Transformer, self).__init__()

        self.mla = MultiLevelAttention(d_model, num_heads, input_vocab_size, rate)

        self.dffnl = DualFeedForwardLayer(d_model, dff, rate)

        self.Classification_layer = tf.keras.layers.Dense(target_categories)

    def call(self, x1, x2, training):

        mla_output = self.mla(x1, x2, training)

        dffnl_output1 = self.dffnl(mla_output, training)

        final_output1 = self.Classification_layer(tf.reduce_mean(dffnl_output1, axis=1))

        final_output1 = tf.nn.softmax(final_output1, axis=-1)

        return final_output1

### Model Summery

In [ ]:
# Instantiate the Transformer model hyperparameters
d_model = 128
num_heads = 4
dff = 128
input_vocab_size = vocabulary_size
target_categories = TotalArticles
rate = 0.1

# Assuming you have instantiated your model
sample_transformer = Transformer(d_model, num_heads, dff, input_vocab_size, target_categories)

# Temp Imputs
batch_size = 32
sequence_length1 = 100
sequence_length2 = 150
dummy_input1 = tf.ones((batch_size, sequence_length1))
dummy_input2 = tf.ones((batch_size, sequence_length2))

# Call the model on the dummy data to build the model
sample_transformer(dummy_input1, dummy_input2)

# Now you can print the summary
sample_transformer.summary()


Model: "transformer_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 multi_level_attention_4 (M  multiple                  746752    
 ultiLevelAttention)                                             
                                                                 
 dual_feed_forward_layer_4   multiple                  33280     
 (DualFeedForwardLayer)                                          
                                                                 
 dense_80 (Dense)            multiple                  39474     
                                                                 
Total params: 819506 (3.13 MB)
Trainable params: 819506 (3.13 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### Instantiate the Transformer model

In [ ]:
transformer_model = Transformer(d_model, num_heads, dff, input_vocab_size, target_categories)

# Classification Regression Transformer Training

## Instantiate the Loss and Accuracy Matrics

In [ ]:
# Define the loss functions for classification and regression tasks
classification_loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

# Define the optimizer
optimizer = tf.keras.optimizers.Adam()

# Define metrics to track during training
classification_loss_metric = tf.keras.metrics.Mean(name='classification_loss')
classification_accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy(name='classification_accuracy')

## Model checkpoint

In [ ]:
checkpoint_path = "/content/drive/MyDrive/.ipynb_checkpoints"
log_file_path = "/content/drive/MyDrive/logging.txt"

ckpt = tf.train.Checkpoint(transformer=transformer_model,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

def save_checkpoint(epoch):
    ckpt_save_path = ckpt_manager.save()
    with open(ckpt_manager.latest_checkpoint + '-epoch.txt', 'w') as f:
        f.write(str(epoch+1))
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,ckpt_save_path))

    with open(log_file_path, 'a') as log_file:
        log_file.write(f"Epoch {epoch + 1}: C-Loss - {classification_loss_metric.result()}, C-Acc - {classification_accuracy_metric.result()}\n")

# Function to load the model and epoch information
def load_checkpoint():
    latest_checkpoint = ckpt_manager.latest_checkpoint
    epoch = 0
    if latest_checkpoint:
        ckpt.restore(latest_checkpoint)
        print('Latest checkpoint restored!!')
        try:
            with open(latest_checkpoint + '-epoch.txt', 'r') as f:
                epoch = int(f.read())
            print(f"Model restored from {latest_checkpoint}, trained up to epoch {epoch}")
        except FileNotFoundError:
            print(f"Epoch information not found for {latest_checkpoint}")
    else:
        print("No checkpoint found.")
    return epoch

In [ ]:
latest_checkpoint_epoch = load_checkpoint()

No checkpoint found.


## Using gradient tape for getting derivatives of loss functions w.r.t. weights then applyiing to optimizer => BACKPROPAGATION

In [ ]:
@tf.function
def train_step(input_data1, input_data2, targets_classification):
    with tf.GradientTape() as tape:
        # Forward pass
        classification_predictions = transformer_model(input_data1, input_data2, training=True)

        # Calculate losses
        classification_loss = classification_loss_fn(targets_classification, classification_predictions)
        total_loss = classification_loss

    # Calculate gradients
    gradients = tape.gradient(total_loss, transformer_model.trainable_variables)

    # Update weights
    optimizer.apply_gradients(zip(gradients, transformer_model.trainable_variables))

    # Update metrics
    classification_loss_metric.update_state(classification_loss)
    classification_accuracy_metric.update_state(targets_classification, classification_predictions)

## Training

In [ ]:
def to_numpy_array(X):
    m = X.str.len().max()
    l = []
    for row in X:
        row = list(row)
        row += [0] * (m - len(row))
        l.append(row)
    return np.array(l)

dummpy_List = []
for qty in y_train_quantity:
    dummpy_List.append([qty])
y_train_quantity = np.array(dummpy_List).astype(np.float32)

In [ ]:
train_data1 = to_numpy_array(X_train_PositionShortText_sequences).astype(np.float32)
train_data2 = to_numpy_array(X_train_LongText_sequences).astype(np.float32)
train_targets_classification = y_train_article_number.to_numpy().astype(np.int32)

# Define hyperparameters and training configurations
num_epochs = 50
batch_size = 32
metrics_names = ['C_loss', 'C_acc']

# Training loop
for epoch in range(latest_checkpoint_epoch,num_epochs):
    # Reset the metrics at the start of each epoch
    classification_loss_metric.reset_states()
    classification_accuracy_metric.reset_states()

    print("\nepoch {}/{}".format(epoch+1,num_epochs))
    pb_i = tf.keras.utils.Progbar(train_data1.shape[0], stateful_metrics=metrics_names)

    # Iterate over the training dataset in batches
    for i in range(0, len(train_data1), batch_size):
        batch_data1 = train_data1[i:i+batch_size]
        batch_data2 = train_data2[i:i+batch_size]
        batch_targets_classification = train_targets_classification[i:i+batch_size]

        # Perform a training step
        train_step(batch_data1, batch_data2, batch_targets_classification)

        values=[('C_loss',classification_loss_metric.result()),
                ('C_acc',classification_accuracy_metric.result())]

        pb_i.add(batch_size, values=values)

    if (epoch+1) % 5 == 0:
        save_checkpoint(epoch)



epoch 1/50


1632/1606 [==============================] - 16s 10ms/step - C_loss: 4.7983 - C_acc: 0.0872

epoch 2/50
1632/1606 [==============================] - 2s 1ms/step - C_loss: 3.1105 - C_acc: 0.2578

epoch 3/50
1632/1606 [==============================] - 2s 1ms/step - C_loss: 2.3768 - C_acc: 0.3244

epoch 4/50
1632/1606 [==============================] - 2s 1ms/step - C_loss: 2.0791 - C_acc: 0.3288

epoch 5/50
1632/1606 [==============================] - 2s 1ms/step - C_loss: 1.9189 - C_acc: 0.3456
Saving checkpoint for epoch 5 at /content/drive/MyDrive/.ipynb_checkpoints/ckpt-1

epoch 6/50
1632/1606 [==============================] - 2s 1ms/step - C_loss: 1.8314 - C_acc: 0.3369

epoch 7/50
1632/1606 [==============================] - 2s 1ms/step - C_loss: 1.7869 - C_acc: 0.3512

epoch 8/50
1632/1606 [==============================] - 2s 1ms/step - C_loss: 1.7605 - C_acc: 0.3425

epoch 9/50
1632/1606 [==============================] - 2s 1ms/step - C_loss: 1.7376 - C_acc: 0.3412

epoch 10/

In [ ]:
test_data1 = to_numpy_array(X_test_PositionShortText_sequences).astype(np.float32)
test_data2 = to_numpy_array(X_test_LongText_sequences).astype(np.float32)
articalno = y_test_article_number.to_numpy().astype(np.int32)

In [ ]:
def Categoricalpredition(data1,data2):
  classification_predictions = transformer_model(data1.reshape(1,-1), data2.reshape(1,-1), training=True)

  # Process the outputs
  # For classification predictions
  classification_predictions = list(classification_predictions.numpy()[0])
  E = max(classification_predictions)*0.70
  class_predictions = []
  for i in range(target_categories-1):
    if classification_predictions[i]>= E:
      class_predictions.append(i)

  return class_predictions

In [ ]:
Categoricalpredition(test_data1[2],test_data2[2])

[82, 180, 200, 203, 208]